## This notebook is a tutorial on estimating the systematic uncertainties associated with various stellar model grids.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import kiauhoku as kh

# data/err are [teff, L/Lsun, [M/H]]
data = [5772, 1, 0]
err = [100, 0.1, 0.1]

### load various interpolators and define log probability functions for mcmc.

In [ ]:
yrec = kh.load_interpolator('yrec')
mist = kh.load_interpolator('mist')
dart = kh.load_interpolator('dartmouth')
gars = kh.load_interpolator('garstec')

In [ ]:
def yrec_prob(theta, grid, data, error):
    model = grid.get_star_eep(theta)
    if model.isnull().any():
        return -np.inf, None

    model_teff = 10**model['Log Teff(K)']
    model_lum = 10**model['L/Lsun']
    model_met = np.log10(model['Zsurf']/model['Xsurf']/0.0253)

    model_params = np.array([model_teff, model_lum, model_met])
    data = np.array(data)
    error = np.array(error)

    chisq = -0.5 * np.sum(((model_params - data)/error)**2)

    model['teff'] = model_teff
    model['luminosity'] = model_lum
    model['metallicity'] = model_met
    model['age'] = model['Age(Gyr)']
    model['lnprob'] = chisq
    
    return chisq, model

def mist_prob(theta, grid, data, error):
    model = grid.get_star_eep(theta)
    if model.isnull().any():
        return -np.inf, None
    
    model_teff = 10**model['log_Teff']
    model_lum = 10**model['log_L']
    model_met = model['log_surf_z'] - np.log10(model['surface_h1']*0.0173)

    model_params = np.array([model_teff, model_lum, model_met])
    data = np.array(data)
    error = np.array(error)

    chisq = -0.5 * np.sum(((model_params - data)/error)**2)

    model['teff'] = model_teff
    model['luminosity'] = model_lum
    model['metallicity'] = model_met
    model['age'] = model['star_age'] / 1e9
    model['lnprob'] = chisq

    return chisq, model

def dart_prob(theta, grid, data, error):
    model = grid.get_star_eep(theta)
    if model.isnull().any():
        return -np.inf, None
    
    model_teff = 10**model['Log T']
    model_lum = 10**model['Log L']
    model_met = np.log10(model['(Z/X)_surf']/0.0229)

    model_params = np.array([model_teff, model_lum, model_met])
    data = np.array(data)
    error = np.array(error)

    chisq = -0.5 * np.sum(((model_params - data)/error)**2)

    model['teff'] = model_teff
    model['luminosity'] = model_lum
    model['metallicity'] = model_met
    model['age'] = model['Age (yrs)'] / 1e9
    model['lnprob'] = chisq

    return chisq, model

def gars_prob(theta, grid, data, error):
    model = grid.get_star_eep(theta)
    if model.isnull().any():
        return -np.inf, None
    
    model_teff = model['Teff']
    model_lum = 10**model['Log L/Lsun']
    model_met = np.log10(model['Zsurf']/model['Xsurf']/0.0245)

    model_params = np.array([model_teff, model_lum, model_met])
    data = np.array(data)
    error = np.array(error)

    chisq = -0.5 * np.sum(((model_params - data)/error)**2)

    model['teff'] = model_teff
    model['luminosity'] = model_lum
    model['metallicity'] = model_met
    model['age'] = model['Age(Myr)'] / 1e3
    model['lnprob'] = chisq

    return chisq, model

### quick mcmc wrapper and computation of model parameter offsets

In [ ]:
def mcmc(grid, lnprob, data, err):
    sampler, chains = grid.mcmc_star(
        lnprob,
        args=(data, err),
        initial_guess=(1, 0, 300),
        guess_width=(0.1, 0.1, 25),
        n_walkers=12,
        n_burnin=100,
        n_iter=10000
    )
    
    return sampler, chains

def compute_offsets(label, ref='yrec'):
    if ref == 'yrec':
        ref_val = yrec_chains[label].median()
    elif ref == 'mist':
        ref_val = mist_chains[label].median()
    elif ref == 'dart':
        ref_val = dart_chains[label].median()
    elif ref == 'gars':
        ref_val = gars_chains[label].median()

    offsets = {
        f'yrec-{ref}': yrec_chains[label].median() - ref_val,
        f'mist-{ref}': mist_chains[label].median() - ref_val,
        f'dart-{ref}': dart_chains[label].median() - ref_val,
        f'gars-{ref}': gars_chains[label].median() - ref_val
    }

    offsets.pop(f'{ref}-{ref}')
    return offsets

### Finally, run mcmc and evaluate offsets.

In [ ]:
yrec_sampler, yrec_chains = mcmc(yrec, yrec_prob, data, err)
mist_sampler, mist_chains = mcmc(mist, mist_prob, data, err)
dart_sampler, dart_chains = mcmc(dart, dart_prob, data, err)
gars_sampler, gars_chains = mcmc(gars, gars_prob, data, err)

print(compute_offsets('age'))
print(compute_offsets('initial_mass'))